In [3]:
import sys

sys.path.append("../../")

In [4]:
import os
from typing import Any, AsyncGenerator, Union
from langchain_aws import ChatBedrock
from app.chatbot import BaseChatbot


class ClaudeSonnetChatbot(BaseChatbot):
    def __init__(self, temperature: float = 0):
        self.llm = ChatBedrock(
            model="us.anthropic.claude-3-5-sonnet-20240620-v1:0",
            model_kwargs={"temperature": temperature},
            credentials_profile_name=os.getenv("AWS_PROFILE"),
            region=os.getenv("AWS_REGION", "us-east-1"),
        )

    def get_text_response(self, prompt: str) -> str:
        response = self.llm.invoke(prompt)
        return response.text()

    async def get_text_response_async(self, prompt: str) -> Union[str, list[Union[str, dict[Any, Any]]]]:
        response = await self.llm.ainvoke(prompt)
        return response.content

    async def stream_response(self, prompt: str) -> AsyncGenerator[Union[str, list[Union[str, dict[Any, Any]]]], None]:
        async for chunk in self.llm.astream(prompt):
            yield chunk.content


# Test Claude 3 Sonnet (supports on-demand throughput)
claude = ClaudeSonnetChatbot(temperature=0.7)
text_response = claude.get_text_response("Explain meaning of life in a single line.")
print("Claude 3 Sonnet text response:", text_response)

print("Claude 3 Sonnet streaming response:")
async for chunk in claude.stream_response("Explain quantum computing in one line"):
    print(chunk, end="")
print()

Claude 3 Sonnet text response: Life's meaning is what you choose to make of it through your actions, relationships, and personal growth.
Claude 3 Sonnet streaming response:
Quantum computing harnesses quantum mechanics to perform certain computations exponentially faster than classical computers.


In [ ]:
from app.common.vector_embedders import LangChainTitanEmbedder
from asyncio import to_thread

embedder = LangChainTitanEmbedder()
result = embedder.embed_single_text("This is great")

result = await to_thread(embedder.embed_single_text, "This is funny")
print(result)

InvalidRequestError: Attribute name 'metadata' is reserved when using the Declarative API.

In [27]:
import json
from uuid import UUID
from pydantic import BaseModel
from langchain.tools import tool

from app.chatbot.chatbot_models import Action


class ArchivalMemorySearchParams(BaseModel):
    """ "Retrieve information from your extensive archival memory into your core working memory."""

    user_id: UUID
    query: str


@tool(
    "archival_memory_search",
    description="Retrieve information from your extensive archival memory into your core working memory.",
    args_schema=ArchivalMemorySearchParams,
    infer_schema=False,
    return_direct=True,
)
def archival_memory_search(input: ArchivalMemorySearchParams):
    return ""


tool = archival_memory_search
print(json.dumps(Action(name=tool.name, description=tool.description, params=tool.args).model_dump()))
print(archival_memory_search.name)
print(archival_memory_search.args)
print(archival_memory_search.description)

{"name": "archival_memory_search", "description": "Retrieve information from your extensive archival memory into your core working memory.", "params": {"user_id": {"format": "uuid", "title": "User Id", "type": "string"}, "query": {"title": "Query", "type": "string"}}}
archival_memory_search
{'user_id': {'format': 'uuid', 'title': 'User Id', 'type': 'string'}, 'query': {'title': 'Query', 'type': 'string'}}
Retrieve information from your extensive archival memory into your core working memory.
